In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import time
import matplotlib.pyplot as plt
%matplotlib inline

/kaggle/input/cricket/Train_sample.csv
/kaggle/input/odi-data/Train.csv
/kaggle/input/odi-data/Test.csv
/kaggle/input/odi-data/Sample_submission.xlsx
/kaggle/input/team-data/team_data.csv


In [2]:
os.chdir('/kaggle/input/odi-data/')
train= pd.read_csv('Train.csv')
test=pd.read_csv('Test.csv')
train['Team1_win']=train.apply(lambda x :1 if x['Team1']==x['MatchWinner'] else 0,axis=1)
train['Team2_win']=train.apply(lambda x :1 if x['Team2']==x['MatchWinner'] else 0,axis=1)
train = train.astype(str)
targets= train['MatchWinner']
train.head()

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch,MatchWinner,Team1_win,Team2_win
0,5,4,37,4,Home,Away,Second,First,Dec,4,0,1
1,1,14,84,7,Neutral,Neutral,First,Second,Sep,1,1,0
2,9,15,47,9,Home,Away,First,Second,Feb,9,1,0
3,7,2,102,6,Home,Away,First,Second,Aug,2,0,1
4,6,8,46,5,Home,Away,First,Second,Aug,6,1,0


# **Feature Engineering**

In [ ]:
height = 15*15
width = 2
team_data = pd.DataFrame(0, index=range(height), columns=range(width))
team_data.columns=['Team1','Team2']
p=1
for x in range (0, height):
    team_data.loc[[x],['Team1']]=int((x/15)+1)
    team_data.loc[[x],['Team2']]=p
    p+=1
    if p>15:
        p=1
team_data=team_data.astype(str)
team_data.head()

In [ ]:
def total_matches(data, team):
    matches=0
    if data['Team1']== team or data['Team2']==team:
        matches +=1
    return matches

def total_wins(data, team):
    wins=0
    if data['Team1']== team and data['Team1_win']=='1':
        wins +=1
    elif data['Team2']== team and data['Team2_win']=='1':
        wins+=1
    return wins

def matches_by_venue(data,team):
    home_matches=0
    away_matches=0
    neutral_matches=0
    if data['Team1']==team:
        if data['Team1_Venue']=='Home':
            home_matches +=1
        elif data['Team1_Venue']=='Away':
            away_matches +=1
        else :
            neutral_matches +=1
    elif data['Team2']==team:
        if data['Team2_Venue']=='Home':
            home_matches +=1
        elif data['Team2_Venue']=='Away':
            away_matches +=1
        else :
            neutral_matches +=1
    return home_matches,away_matches,neutral_matches

def total_home_matches(data,team):
    p=matches_by_venue(data,team)
    return p[0]
def total_away_matches(data,team):
    p=matches_by_venue(data,team)
    return p[1]
def total_neutral_matches(data,team):
    p=matches_by_venue(data,team)
    return p[2]
    

In [ ]:
# create dictionary with data for each team
total_teams=15
teams=[]
total_matches_of_team=[]
total_wins_of_team=[]
win_frac=[]
total_home_matches_of_team=[]
total_away_matches_of_team=[]
total_neutral_matches_of_team=[]

for x in range (1,total_teams+1):
    teams.append(x)
    matches=train.apply(lambda y : total_matches(y, str(x)), axis=1)
    total_matches_of_team.append(sum(matches))
    wins=train.apply(lambda y : total_wins(y, str(x)), axis=1)
    total_wins_of_team.append(sum(wins))
    home_matches=train.apply(lambda y : total_home_matches(y, str(x)), axis=1)
    total_home_matches_of_team.append(sum(home_matches))
    away_matches=train.apply(lambda y : total_away_matches(y, str(x)), axis=1)
    total_away_matches_of_team.append(sum(away_matches))
    neutral_matches=train.apply(lambda y : total_neutral_matches(y, str(x)), axis=1)
    total_neutral_matches_of_team.append(sum(neutral_matches))
    
total_matches_dict= {teams[i]: total_matches_of_team[i] for i in range(len(teams))} 
total_wins_dict= {teams[i]: total_wins_of_team[i] for i in range(len(teams))}
total_home_dict= {teams[i]: total_home_matches_of_team[i] for i in range(len(teams))}
total_away_dict= {teams[i]: total_away_matches_of_team[i] for i in range(len(teams))} 
total_neutral_dict= {teams[i]: total_neutral_matches_of_team[i] for i in range(len(teams))} 
win_frac_dict={teams[i]: total_wins_of_team[i]/total_matches_of_team[i] for i in range(len(teams))} 

In [ ]:
# assign data to df from created dictionary
def assign_data(data, property_, dict_):
    for key in dict_:
        if data[property_]== str(key):
            return dict_[key]

# assign dictionary values to df
team_data['Team1_total_matches']=team_data.apply(lambda x : assign_data(x, 'Team1', total_matches_dict) , axis=1)
team_data['Team2_total_matches']=team_data.apply(lambda x : assign_data(x, 'Team2', total_matches_dict) , axis=1)
team_data['Team1_total_wins']=team_data.apply(lambda x : assign_data(x, 'Team1', total_wins_dict) , axis=1)
team_data['Team2_total_wins']=team_data.apply(lambda x : assign_data(x, 'Team2', total_wins_dict) , axis=1)
team_data['Team1_total_matches_home']=team_data.apply(lambda x : assign_data(x, 'Team1', total_home_dict) , axis=1)
team_data['Team2_total_matches_home']=team_data.apply(lambda x : assign_data(x, 'Team2', total_home_dict) , axis=1)
team_data['Team1_total_matches_away']=team_data.apply(lambda x : assign_data(x, 'Team1', total_away_dict) , axis=1)
team_data['Team2_total_matches_away']=team_data.apply(lambda x : assign_data(x, 'Team2', total_away_dict) , axis=1)
team_data['Team1_total_matches_neutral']=team_data.apply(lambda x : assign_data(x, 'Team1', total_neutral_dict) , axis=1)
team_data['Team2_total_matches_neutral']=team_data.apply(lambda x : assign_data(x, 'Team2', total_neutral_dict) , axis=1)
team_data['Team1_total_win_frac']=team_data.apply(lambda x : assign_data(x, 'Team1', win_frac_dict) , axis=1)
team_data['Team2_total_win_frac']=team_data.apply(lambda x : assign_data(x, 'Team2', win_frac_dict) , axis=1)
team_data.head()

In [ ]:
def matches_against_team(data, team1, team2):
    total_matches=0
    total_matches_home=0
    total_matches_away=0
    total_matches_neutral =0
    total_wins=0
    total_wins_home=0
    total_wins_away=0
    total_wins_neutral =0
    if data['Team1']==str(team1) and data['Team2']==str(team2):
        total_matches +=1
        if data['Team1_Venue']=='Home':
            total_matches_home +=1
            if data['MatchWinner']==str(team1):
                total_wins_home+=1
                
        elif data['Team1_Venue']=='Away':
            total_matches_away +=1
            if data['MatchWinner']==team1:
                total_wins_away+=1
        elif data['Team1_Venue']=='Neutral':
            total_matches_neutral +=1
            if data['MatchWinner']==team1:
                total_wins_neutral+=1
            
        if data['MatchWinner']==team1:
            total_wins +=1

            
    elif (data['Team1']==team2) & (data['Team2']==team1):
        total_matches +=1
        if data['Team2_Venue']=='Home':
            total_matches_home +=1
            if data['MatchWinner']==team1:
                total_wins_home+=1
        elif data['Team2_Venue']=='Away':
            total_matches_away +=1
            if data['MatchWinner']==team1:
                total_wins_away+=1
        elif data['Team2_Venue']=='Neutral':
            total_matches_neutral +=1
            if data['MatchWinner']==team1:
                total_wins_neutral+=1
        if data['MatchWinner']==team1:
            total_wins +=1
    return total_matches, total_matches_home,total_matches_away,total_matches_neutral,total_wins,total_wins_home,total_wins_away,total_wins_neutral
            
def total_matches_vs(data, team1, team2):
    p=matches_against_team(data, team1, team2)
    return p[0]
def total_matches_home_vs(data, team1, team2):
    p=matches_against_team(data, team1, team2)
    return p[1]
def total_matches_away_vs(data, team1, team2):
    p=matches_against_team(data, team1, team2)
    return p[2]
def total_matches_neutral_vs(data, team1, team2):
    p=matches_against_team(data, team1, team2)
    return p[3]
def total_wins_vs(data, team1, team2):
    p=matches_against_team(data, team1, team2)
    return p[4]
def total_wins_home_vs(data, team1, team2):
    p=matches_against_team(data, team1, team2)
    return p[5]
def total_wins_away_vs(data, team1, team2):
    p=matches_against_team(data, team1, team2)
    return p[6]
def total_wins_neutral_vs(data, team1, team2):
    p=matches_against_team(data, team1, team2)
    return p[7]

In [ ]:
start = time.time()
team_data['total_matches_1vs2']=team_data.apply(lambda x : sum(train.apply(lambda y : total_matches_vs(y, x['Team1'],x['Team2']),axis=1)), axis=1)
team_data['total_matches_1vs2_home']=team_data.apply(lambda x : sum(train.apply(lambda y : total_matches_home_vs(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
team_data['total_matches_1vs2_away']=team_data.apply(lambda x : sum(train.apply(lambda y : total_matches_away_vs(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
team_data['total_matches_1vs2_neutral']=team_data.apply(lambda x : sum(train.apply(lambda y : total_matches_neutral_vs(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
team_data['total_wins_1vs2']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_vs(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
team_data['total_wins_1vs2_home']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_home_vs(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
team_data['total_wins_1vs2_away']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_away_vs(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
team_data['total_wins_1vs2_neutral']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_neutral_vs(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
team_data['total_wins_2vs1']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_vs(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
team_data['total_wins_2vs1_away']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_away_vs(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
team_data['total_wins_2vs1_home']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_home_vs(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
team_data['total_wins_2vs1_neutral']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_neutral_vs(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [ ]:
def win_by_innings(data, team1, team2):
    total_wins_bat_first=0
    total_wins_bat_second=0
    total_wins_home_bat_first=0
    total_wins_home_bat_second=0
    if data['Team1']==str(team1) and data['Team2']==str(team2):
        if data['MatchWinner']==str(team1):
            if data['Team1_Innings']=='First':
                total_wins_bat_first +=1
                if data['Team1_Venue']=='Home':
                    total_wins_home_bat_first +=1
            elif data['Team1_Innings']=='Second':
                total_wins_bat_second +=1
                if data['Team1_Venue']=='Home':
                    total_wins_home_bat_second +=1

            
    elif data['Team1']==team2 and data['Team2']==team1:
        if data['MatchWinner']==str(team1):
            if data['Team2_Innings']=='First':
                total_wins_bat_first +=1
                if data['Team2_Venue']=='Home':
                    total_wins_home_bat_first +=1
            elif data['Team2_Innings']=='Second':
                total_wins_bat_second +=1
                if data['Team2_Venue']=='Home':
                    total_wins_home_bat_second +=1
    return total_wins_bat_first, total_wins_bat_second,total_wins_home_bat_first,total_wins_home_bat_second


def total_wins_bat_first(data, team1, team2):
    p=win_by_innings(data, team1, team2)
    return p[0]
def total_wins_bat_second(data, team1, team2):
    p=win_by_innings(data, team1, team2)
    return p[1]
def total_wins_home_bat_first(data, team1, team2):
    p=win_by_innings(data, team1, team2)
    return p[2]

def total_wins_home_bat_second(data, team1, team2):
    p=win_by_innings(data, team1, team2)
    return p[3]

In [ ]:
def win_by_innings1(data, team1, team2):
    total_wins_bat_first=0
    total_wins_bat_second=0
    total_wins_home_bat_first=0
    total_wins_away_bat_first=0
    total_wins_neutral_bat_first =0
    total_wins_home_bat_second=0
    total_wins_away_bat_second=0
    total_wins_neutral_bat_second =0
    if data['Team1_Venue']=='Away' and data['Team1']==str(team1) and data['Team2']==str(team2) and data['Team1_Innings']=='First' and data['MatchWinner']==str(team1):
        total_wins_away_bat_first +=1
    if data['Team2_Venue']=='Away' and data['Team2']==str(team1) and data['Team1']==str(team2) and data['Team2_Innings']=='First' and data['MatchWinner']==str(team1):
        total_wins_away_bat_first +=1
    return total_wins_away_bat_first



def total_wins_away_bat_first(data, team1, team2):
    p=win_by_innings1(data, team1, team2)
    return p

def win_by_innings2(data, team1, team2):
    total_wins_bat_first=0
    total_wins_bat_second=0
    total_wins_home_bat_first=0
    total_wins_away_bat_first=0
    total_wins_neutral_bat_first =0
    total_wins_home_bat_second=0
    total_wins_away_bat_second=0
    total_wins_neutral_bat_second =0
    if data['Team1_Venue']=='Neutral' and data['Team1']==str(team1) and data['Team2']==str(team2) and data['Team1_Innings']=='First' and data['MatchWinner']==str(team1):
        total_wins_neutral_bat_first +=1
    if data['Team2_Venue']=='Neutral' and data['Team2']==str(team1) and data['Team1']==str(team2) and data['Team2_Innings']=='First' and data['MatchWinner']==str(team1):
        total_wins_neutral_bat_first +=1
    return total_wins_neutral_bat_first



def total_wins_neutral_bat_first(data, team1, team2):
    p=win_by_innings2(data, team1, team2)
    return p




def win_by_innings3(data, team1, team2):
    total_wins_bat_first=0
    total_wins_bat_second=0
    total_wins_home_bat_first=0
    total_wins_away_bat_first=0
    total_wins_neutral_bat_first =0
    total_wins_home_bat_second=0
    total_wins_away_bat_second=0
    total_wins_neutral_bat_second =0
    if data['Team1_Venue']=='Away' and data['Team1']==str(team1) and data['Team2']==str(team2) and data['Team1_Innings']=='Second' and data['MatchWinner']==str(team1):
        total_wins_away_bat_second +=1
    if data['Team2_Venue']=='Away' and data['Team2']==str(team1) and data['Team1']==str(team2) and data['Team2_Innings']=='Second' and data['MatchWinner']==str(team1):
        total_wins_away_bat_second +=1
    return total_wins_away_bat_second



def total_wins_away_bat_second(data, team1, team2):
    p=win_by_innings3(data, team1, team2)
    return p

def win_by_innings4(data, team1, team2):
    total_wins_bat_first=0
    total_wins_bat_second=0
    total_wins_home_bat_first=0
    total_wins_away_bat_first=0
    total_wins_neutral_bat_first =0
    total_wins_home_bat_second=0
    total_wins_away_bat_second=0
    total_wins_neutral_bat_second =0
    if data['Team1_Venue']=='Neutral' and data['Team1']==str(team1) and data['Team2']==str(team2) and data['Team1_Innings']=='Second' and data['MatchWinner']==str(team1):
        total_wins_neutral_bat_second +=1
    if data['Team2_Venue']=='Neutral' and data['Team2']==str(team1) and data['Team1']==str(team2) and data['Team2_Innings']=='Second' and data['MatchWinner']==str(team1):
        total_wins_neutral_bat_second +=1
    return total_wins_neutral_bat_second



def total_wins_neutral_bat_second(data, team1, team2):
    p=win_by_innings4(data, team1, team2)
    return p

In [ ]:
start = time.time()
team_data['total_wins_bat_first_1']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_bat_first(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
end = time.time()
print(end - start)


In [ ]:
start = time.time()
team_data['total_wins_bat_second_1']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_bat_second(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
end = time.time()
print(end - start)


In [ ]:
start = time.time()
team_data['total_wins_bat_first_2']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_bat_first(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
end = time.time()
print(end - start)



In [ ]:
start = time.time()
team_data['total_wins_bat_second_2']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_bat_second(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
team_data['total_wins_bat_first_home_1']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_home_bat_first(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
team_data['total_wins_bat_first_home_2']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_home_bat_first(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
team_data['total_wins_bat_first_away_1']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_away_bat_first(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
end = time.time()
print(end - start)
team_data.head()

In [ ]:
start = time.time()
team_data['total_wins_bat_first_away_2']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_away_bat_first(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
end = time.time()
print(end - start)
team_data.head()

In [ ]:
start = time.time()
team_data['total_wins_bat_first_neutral_1']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_neutral_bat_first(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
team_data['total_wins_bat_first_neutral_2']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_neutral_bat_first(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
end = time.time()
print(end - start)
team_data.head()

In [ ]:
#total_wins_home_bat_second
start = time.time()
team_data['total_wins_bat_second_home_1']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_home_bat_second(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
team_data['total_wins_bat_second_home_2']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_home_bat_second(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
team_data['total_wins_bat_second_away_1']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_away_bat_second(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
team_data['total_wins_bat_second_away_2']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_away_bat_second(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
team_data['total_wins_bat_second_neutral_1']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_neutral_bat_second(y, str(x['Team1']),str(x['Team2'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
team_data['total_wins_bat_second_neutral_2']=team_data.apply(lambda x : sum(train.apply(lambda y : total_wins_neutral_bat_second(y, str(x['Team2']),str(x['Team1'])),axis=1))  , axis=1)
end = time.time()
print(end - start)

In [3]:
targets= train['MatchWinner']
train.drop(['MatchWinner','Team1_win','Team2_win'], axis=1, inplace=True)
train.head()

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch
0,5,4,37,4,Home,Away,Second,First,Dec
1,1,14,84,7,Neutral,Neutral,First,Second,Sep
2,9,15,47,9,Home,Away,First,Second,Feb
3,7,2,102,6,Home,Away,First,Second,Aug
4,6,8,46,5,Home,Away,First,Second,Aug


In [4]:
all_data=pd.concat([train, test])
#team_data=team_data.astype(str)
all_data=all_data.astype(str)

In [ ]:
#team_data.to_csv('team_data.csv', index=False)
team_data=pd.read_csv('/kaggle/input/team-data/team_data.csv')
team_data.head()
team_data=team_data.astype(float)
team_data['Team1']=team_data['Team1'].astype(int)
team_data['Team2']=team_data['Team2'].astype(int)
team_data['Team1']=team_data['Team1'].astype(str)
team_data['Team2']=team_data['Team2'].astype(str)
team_data.dtypes

Now lets define the model!

I will use catboost of xgboost initially using label ecoder and not one hot ecoding as it will take a longer time to train.
once i get the predictions from label encoder, i will try one hot encoding and compare the results!

In [7]:
num_features= ['Team1_total_matches', 'Team2_total_matches', 'Team1_total_wins',
       'Team2_total_wins', 'Team1_total_matches_home',
       'Team2_total_matches_home', 'Team1_total_matches_away',
       'Team2_total_matches_away', 'Team1_total_matches_neutral',
       'Team2_total_matches_neutral', 'Team1_total_win_frac',
       'Team2_total_win_frac', 'total_matches_1vs2', 'total_matches_1vs2_home',
       'total_matches_1vs2_away', 'total_matches_1vs2_neutral',
       'total_wins_1vs2', 'total_wins_1vs2_home', 'total_wins_1vs2_away',
       'total_wins_1vs2_neutral', 'total_wins_2vs1', 'total_wins_2vs1_away',
       'total_wins_2vs1_home', 'total_wins_2vs1_neutral',
       'total_wins_bat_first_1', 'total_wins_bat_second_1',
       'total_wins_bat_first_2', 'total_wins_bat_second_2',
       'total_wins_bat_first_home_1', 'total_wins_bat_first_home_2',
       'total_wins_bat_first_away_1', 'total_wins_bat_first_away_2',
       'total_wins_bat_first_neutral_1', 'total_wins_bat_first_neutral_2',
       'total_wins_bat_second_home_1', 'total_wins_bat_second_home_2',
       'total_wins_bat_second_away_1', 'total_wins_bat_second_neutral_2',
       'total_wins_bat_second_away_2', 'total_wins_bat_second_neutral_1']

In [8]:
cat_features=['Team1', 'Team2','Stadium', 'HostCountry', 'Team1_Venue',
       'Team2_Venue', 'Team1_Innings', 'Team2_Innings', 'MonthOfMatch',
       ] 

In [15]:
import time
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
os.chdir('/kaggle/working')
from catboost import CatBoostClassifier

train_acc=[]
test_acc=[]
for x in range (0, len(num_features)):
    new_df = pd.merge(all_data, team_data,  how='left', left_on=['Team1','Team2'], right_on = ['Team1','Team2'])
    new_df=new_df.reset_index(drop=True)
    df_encoded=new_df
    le=LabelEncoder()
    scaler = StandardScaler()
    #df_encoded=all_data

    for num in num_features:
        df_encoded[num]=scaler.fit_transform(df_encoded[num].to_numpy().reshape(-1,1))
    

    for cat in cat_features:
        df_encoded[cat]=le.fit_transform(df_encoded[cat].astype(str))
    
    f_to_remove=num_features[x]
    df_encoded.drop(f_to_remove, axis=1, inplace=True)
    traindata=df_encoded[:targets.shape[0]]
    testdata= df_encoded[targets.shape[0]:]
    cat_features_indices=np.where(traindata.dtypes != np.float)[0]
    
    X_train, X_test, Y_train, Y_test= train_test_split(traindata,targets, test_size=0.3, random_state=42)
    model = CatBoostClassifier(iterations=100,
                           learning_rate=0.01,
                           depth=10,
                           loss_function="MultiClass",od_type = "Iter",task_type="GPU",random_seed=42,l2_leaf_reg=1)
    # Fit model
    model.fit(X_train,Y_train,early_stopping_rounds=10, verbose=True, cat_features=cat_features_indices)

    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [value for value in y_pred]# evaluate predictions
    accuracy = accuracy_score(Y_test, predictions)
    # make predictions for train data
    y_pred_train = model.predict(X_train)
    predictions1 = [value for value in y_pred_train]# evaluate predictions
    accuracy1 = accuracy_score(Y_train, predictions1)
    print("Test_Accuracy: %.2f%%" % (accuracy * 100.0))
    print("Train_Accuracy: %.2f%%" % (accuracy1 * 100.0))
    train_acc.append(accuracy1)
    test_acc.append(accuracy)


# features to drop based on test accuracy results
l=[x+1 for x in range (0,len(test_acc))]

test_acc_dict= {l[i]: test_acc[i] for i in range(len(test_acc))} 

def f_to_drop(test_acc_dict):
    f_to_drop_list=[]
    if test_acc_dict[1] <62:
        f_to_drop_list.append(1)
    
    for x in range (1,len(test_acc_dict)):
        if test_acc_dict[x+1] > test_acc_dict[x]:
            f_to_drop_list.append(x+1)
    return f_to_drop_list
        
f_to_drop_list=f_to_drop(test_acc_dict)
f_to_drop_list=[num_features[x-1] for x in f_to_drop_list]

In [63]:
new_df = pd.merge(all_data, team_data,  how='left', left_on=['Team1','Team2'], right_on = ['Team1','Team2'])
new_df=new_df.reset_index(drop=True)
df_encoded=new_df
le=LabelEncoder()
scaler = StandardScaler()
#df_encoded=all_data

for num in num_features:
    df_encoded[num]=scaler.fit_transform(df_encoded[num].to_numpy().reshape(-1,1))
    

for cat in cat_features:
    df_encoded[cat]=le.fit_transform(df_encoded[cat].astype(str))
    
#f_to_remove=['total_wins_bat_first_neutral_1','MonthOfMatch','HostCountry']

#df_encoded.drop(f_to_remove, axis=1, inplace=True)
traindata=df_encoded[:targets.shape[0]]
testdata= df_encoded[targets.shape[0]:]

cat_features_indices=np.where(traindata.dtypes != np.float)[0]
cat_features_indices

array([0, 1, 2, 3, 4, 5, 6])

In [67]:
# split into train and test set
X_train, X_test, Y_train, Y_test= train_test_split(traindata,targets, test_size=0.3, random_state=42)
os.chdir('/kaggle/working')
# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=10,
                           loss_function="MultiClass",od_type = "Iter",task_type="GPU",random_seed=42,l2_leaf_reg=1)
# Fit model
model.fit(X_train,Y_train,early_stopping_rounds=10, verbose=True, cat_features=cat_features_indices)


# Get predicted classes
#preds_class = model.predict(X_test)
# Get predicted probabilities for each class
#preds_proba = model.predict_proba(X_test)
# Get predicted RawFormulaVal
#preds_raw = model.X_test(X_test, 
                   #       prediction_type='RawFormulaVal')

0:	learn: 2.7488512	total: 132ms	remaining: 2m 11s
1:	learn: 2.7085295	total: 253ms	remaining: 2m 6s
2:	learn: 2.6635856	total: 377ms	remaining: 2m 5s
3:	learn: 2.6270168	total: 496ms	remaining: 2m 3s
4:	learn: 2.5851760	total: 615ms	remaining: 2m 2s
5:	learn: 2.5482169	total: 732ms	remaining: 2m 1s
6:	learn: 2.5139061	total: 876ms	remaining: 2m 4s
7:	learn: 2.4811354	total: 1.04s	remaining: 2m 8s
8:	learn: 2.4507070	total: 1.16s	remaining: 2m 8s
9:	learn: 2.4182461	total: 1.28s	remaining: 2m 6s
10:	learn: 2.3899247	total: 1.39s	remaining: 2m 5s
11:	learn: 2.3591886	total: 1.51s	remaining: 2m 4s
12:	learn: 2.3295185	total: 1.62s	remaining: 2m 3s
13:	learn: 2.3010770	total: 1.74s	remaining: 2m 2s
14:	learn: 2.2735436	total: 1.89s	remaining: 2m 4s
15:	learn: 2.2517055	total: 2.06s	remaining: 2m 6s
16:	learn: 2.2271871	total: 2.22s	remaining: 2m 8s
17:	learn: 2.2045790	total: 2.43s	remaining: 2m 12s
18:	learn: 2.1801239	total: 2.55s	remaining: 2m 11s
19:	learn: 2.1569010	total: 2.67s	rema

160:	learn: 1.0208812	total: 19.8s	remaining: 1m 43s
161:	learn: 1.0179420	total: 19.9s	remaining: 1m 43s
162:	learn: 1.0146270	total: 20.1s	remaining: 1m 42s
163:	learn: 1.0109223	total: 20.2s	remaining: 1m 42s
164:	learn: 1.0079031	total: 20.3s	remaining: 1m 42s
165:	learn: 1.0048485	total: 20.4s	remaining: 1m 42s
166:	learn: 1.0017182	total: 20.5s	remaining: 1m 42s
167:	learn: 0.9987086	total: 20.8s	remaining: 1m 42s
168:	learn: 0.9955088	total: 21s	remaining: 1m 43s
169:	learn: 0.9925923	total: 21.2s	remaining: 1m 43s
170:	learn: 0.9893322	total: 21.5s	remaining: 1m 43s
171:	learn: 0.9862736	total: 21.6s	remaining: 1m 43s
172:	learn: 0.9830285	total: 21.7s	remaining: 1m 43s
173:	learn: 0.9800073	total: 21.8s	remaining: 1m 43s
174:	learn: 0.9769853	total: 21.9s	remaining: 1m 43s
175:	learn: 0.9740787	total: 22.1s	remaining: 1m 43s
176:	learn: 0.9712397	total: 22.2s	remaining: 1m 43s
177:	learn: 0.9678538	total: 22.3s	remaining: 1m 43s
178:	learn: 0.9647749	total: 22.5s	remaining: 1m

316:	learn: 0.6881885	total: 40.4s	remaining: 1m 27s
317:	learn: 0.6867318	total: 40.6s	remaining: 1m 26s
318:	learn: 0.6854325	total: 40.7s	remaining: 1m 26s
319:	learn: 0.6841903	total: 40.8s	remaining: 1m 26s
320:	learn: 0.6827965	total: 40.9s	remaining: 1m 26s
321:	learn: 0.6814391	total: 41s	remaining: 1m 26s
322:	learn: 0.6800172	total: 41.1s	remaining: 1m 26s
323:	learn: 0.6786252	total: 41.3s	remaining: 1m 26s
324:	learn: 0.6773524	total: 41.4s	remaining: 1m 25s
325:	learn: 0.6760041	total: 41.5s	remaining: 1m 25s
326:	learn: 0.6746856	total: 41.6s	remaining: 1m 25s
327:	learn: 0.6733591	total: 41.7s	remaining: 1m 25s
328:	learn: 0.6719217	total: 41.9s	remaining: 1m 25s
329:	learn: 0.6704924	total: 42s	remaining: 1m 25s
330:	learn: 0.6693414	total: 42.1s	remaining: 1m 25s
331:	learn: 0.6681652	total: 42.2s	remaining: 1m 24s
332:	learn: 0.6670272	total: 42.3s	remaining: 1m 24s
333:	learn: 0.6657843	total: 42.4s	remaining: 1m 24s
334:	learn: 0.6645874	total: 42.6s	remaining: 1m 2

472:	learn: 0.5299123	total: 59.9s	remaining: 1m 6s
473:	learn: 0.5291369	total: 1m	remaining: 1m 6s
474:	learn: 0.5283864	total: 1m	remaining: 1m 6s
475:	learn: 0.5277110	total: 1m	remaining: 1m 6s
476:	learn: 0.5268819	total: 1m	remaining: 1m 6s
477:	learn: 0.5261644	total: 1m	remaining: 1m 6s
478:	learn: 0.5254136	total: 1m	remaining: 1m 5s
479:	learn: 0.5247400	total: 1m	remaining: 1m 5s
480:	learn: 0.5239634	total: 1m	remaining: 1m 5s
481:	learn: 0.5231957	total: 1m 1s	remaining: 1m 5s
482:	learn: 0.5225868	total: 1m 1s	remaining: 1m 5s
483:	learn: 0.5217682	total: 1m 1s	remaining: 1m 5s
484:	learn: 0.5211853	total: 1m 1s	remaining: 1m 5s
485:	learn: 0.5205359	total: 1m 1s	remaining: 1m 5s
486:	learn: 0.5197964	total: 1m 1s	remaining: 1m 4s
487:	learn: 0.5191796	total: 1m 1s	remaining: 1m 4s
488:	learn: 0.5185408	total: 1m 1s	remaining: 1m 4s
489:	learn: 0.5178797	total: 1m 1s	remaining: 1m 4s
490:	learn: 0.5170604	total: 1m 2s	remaining: 1m 4s
491:	learn: 0.5163215	total: 1m 2s	r

630:	learn: 0.4370673	total: 1m 19s	remaining: 46.7s
631:	learn: 0.4366243	total: 1m 20s	remaining: 46.6s
632:	learn: 0.4361724	total: 1m 20s	remaining: 46.5s
633:	learn: 0.4358332	total: 1m 20s	remaining: 46.3s
634:	learn: 0.4353188	total: 1m 20s	remaining: 46.2s
635:	learn: 0.4349198	total: 1m 20s	remaining: 46.2s
636:	learn: 0.4344031	total: 1m 20s	remaining: 46.1s
637:	learn: 0.4339217	total: 1m 21s	remaining: 46s
638:	learn: 0.4335468	total: 1m 21s	remaining: 45.9s
639:	learn: 0.4331118	total: 1m 21s	remaining: 45.8s
640:	learn: 0.4325910	total: 1m 21s	remaining: 45.7s
641:	learn: 0.4320324	total: 1m 21s	remaining: 45.5s
642:	learn: 0.4316117	total: 1m 21s	remaining: 45.4s
643:	learn: 0.4311058	total: 1m 21s	remaining: 45.3s
644:	learn: 0.4304903	total: 1m 22s	remaining: 45.1s
645:	learn: 0.4300420	total: 1m 22s	remaining: 45s
646:	learn: 0.4296097	total: 1m 22s	remaining: 44.9s
647:	learn: 0.4291184	total: 1m 22s	remaining: 44.8s
648:	learn: 0.4287516	total: 1m 22s	remaining: 44.

787:	learn: 0.3732664	total: 1m 40s	remaining: 26.9s
788:	learn: 0.3729582	total: 1m 40s	remaining: 26.8s
789:	learn: 0.3725416	total: 1m 40s	remaining: 26.7s
790:	learn: 0.3722776	total: 1m 40s	remaining: 26.5s
791:	learn: 0.3719291	total: 1m 40s	remaining: 26.4s
792:	learn: 0.3716316	total: 1m 40s	remaining: 26.3s
793:	learn: 0.3713031	total: 1m 40s	remaining: 26.2s
794:	learn: 0.3710453	total: 1m 40s	remaining: 26s
795:	learn: 0.3706737	total: 1m 41s	remaining: 25.9s
796:	learn: 0.3703646	total: 1m 41s	remaining: 25.8s
797:	learn: 0.3700700	total: 1m 41s	remaining: 25.7s
798:	learn: 0.3696961	total: 1m 41s	remaining: 25.5s
799:	learn: 0.3693346	total: 1m 41s	remaining: 25.4s
800:	learn: 0.3689959	total: 1m 41s	remaining: 25.3s
801:	learn: 0.3686618	total: 1m 41s	remaining: 25.1s
802:	learn: 0.3682806	total: 1m 41s	remaining: 25s
803:	learn: 0.3679553	total: 1m 42s	remaining: 24.9s
804:	learn: 0.3676549	total: 1m 42s	remaining: 24.8s
805:	learn: 0.3672664	total: 1m 42s	remaining: 24.

944:	learn: 0.3278927	total: 1m 59s	remaining: 6.96s
945:	learn: 0.3276793	total: 1m 59s	remaining: 6.84s
946:	learn: 0.3273715	total: 1m 59s	remaining: 6.71s
947:	learn: 0.3271464	total: 2m	remaining: 6.58s
948:	learn: 0.3268644	total: 2m	remaining: 6.46s
949:	learn: 0.3266182	total: 2m	remaining: 6.33s
950:	learn: 0.3263468	total: 2m	remaining: 6.2s
951:	learn: 0.3260485	total: 2m	remaining: 6.08s
952:	learn: 0.3257871	total: 2m	remaining: 5.95s
953:	learn: 0.3254582	total: 2m	remaining: 5.83s
954:	learn: 0.3251906	total: 2m	remaining: 5.7s
955:	learn: 0.3249033	total: 2m 1s	remaining: 5.57s
956:	learn: 0.3246387	total: 2m 1s	remaining: 5.45s
957:	learn: 0.3244160	total: 2m 1s	remaining: 5.32s
958:	learn: 0.3241145	total: 2m 1s	remaining: 5.19s
959:	learn: 0.3238372	total: 2m 1s	remaining: 5.07s
960:	learn: 0.3235116	total: 2m 1s	remaining: 4.94s
961:	learn: 0.3232914	total: 2m 1s	remaining: 4.82s
962:	learn: 0.3230315	total: 2m 2s	remaining: 4.69s
963:	learn: 0.3227440	total: 2m 2s	

In [71]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [value for value in y_pred]# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
# make predictions for train data
y_pred_train = model.predict(X_train)
predictions1 = [value for value in y_pred_train]# evaluate predictions
accuracy1 = accuracy_score(Y_train, predictions1)
print("Test_Accuracy: %.2f%%" % (accuracy * 100.0))
print("Train_Accuracy: %.2f%%" % (accuracy1 * 100.0))

Test_Accuracy: 74.24%
Train_Accuracy: 75.33%


In [70]:
#fit on whole data and make predictions
model.fit(traindata,targets,early_stopping_rounds=10, verbose=True, cat_features=cat_features_indices)
preds_final = model.predict_proba(testdata)
preds_df=pd.DataFrame(preds_final)
os.chdir('/kaggle/working/')
to_submit=preds_df.to_excel('predictions.xlsx', index=False)

0:	learn: 2.7469097	total: 133ms	remaining: 2m 13s
1:	learn: 2.7323740	total: 255ms	remaining: 2m 7s
2:	learn: 2.6832223	total: 399ms	remaining: 2m 12s
3:	learn: 2.6369520	total: 515ms	remaining: 2m 8s
4:	learn: 2.5960138	total: 629ms	remaining: 2m 5s
5:	learn: 2.5582166	total: 743ms	remaining: 2m 3s
6:	learn: 2.5196893	total: 858ms	remaining: 2m 1s
7:	learn: 2.4870333	total: 972ms	remaining: 2m
8:	learn: 2.4550925	total: 1.09s	remaining: 1m 59s
9:	learn: 2.4191090	total: 1.2s	remaining: 1m 58s
10:	learn: 2.3844582	total: 1.32s	remaining: 1m 58s
11:	learn: 2.3516326	total: 1.43s	remaining: 1m 57s
12:	learn: 2.3200974	total: 1.55s	remaining: 1m 57s
13:	learn: 2.2919155	total: 1.66s	remaining: 1m 56s
14:	learn: 2.2645051	total: 1.78s	remaining: 1m 56s
15:	learn: 2.2365559	total: 1.89s	remaining: 1m 56s
16:	learn: 2.2098159	total: 2s	remaining: 1m 55s
17:	learn: 2.1836878	total: 2.12s	remaining: 1m 55s
18:	learn: 2.1583669	total: 2.23s	remaining: 1m 55s
19:	learn: 2.1355619	total: 2.35s	r

158:	learn: 1.0058300	total: 19.2s	remaining: 1m 41s
159:	learn: 1.0029473	total: 19.3s	remaining: 1m 41s
160:	learn: 0.9997575	total: 19.5s	remaining: 1m 41s
161:	learn: 0.9968355	total: 19.6s	remaining: 1m 41s
162:	learn: 0.9941731	total: 19.7s	remaining: 1m 41s
163:	learn: 0.9910024	total: 19.8s	remaining: 1m 41s
164:	learn: 0.9877713	total: 19.9s	remaining: 1m 40s
165:	learn: 0.9854729	total: 20.1s	remaining: 1m 40s
166:	learn: 0.9828947	total: 20.2s	remaining: 1m 40s
167:	learn: 0.9795007	total: 20.3s	remaining: 1m 40s
168:	learn: 0.9766260	total: 20.4s	remaining: 1m 40s
169:	learn: 0.9736673	total: 20.6s	remaining: 1m 40s
170:	learn: 0.9707143	total: 20.7s	remaining: 1m 40s
171:	learn: 0.9676042	total: 20.8s	remaining: 1m 40s
172:	learn: 0.9652005	total: 20.9s	remaining: 1m 39s
173:	learn: 0.9627255	total: 21s	remaining: 1m 39s
174:	learn: 0.9600176	total: 21.2s	remaining: 1m 39s
175:	learn: 0.9572829	total: 21.3s	remaining: 1m 39s
176:	learn: 0.9545411	total: 21.4s	remaining: 1m

314:	learn: 0.7072535	total: 38.5s	remaining: 1m 23s
315:	learn: 0.7059839	total: 38.7s	remaining: 1m 23s
316:	learn: 0.7048548	total: 38.9s	remaining: 1m 23s
317:	learn: 0.7036157	total: 39s	remaining: 1m 23s
318:	learn: 0.7022963	total: 39.2s	remaining: 1m 23s
319:	learn: 0.7011960	total: 39.3s	remaining: 1m 23s
320:	learn: 0.7001316	total: 39.4s	remaining: 1m 23s
321:	learn: 0.6991099	total: 39.5s	remaining: 1m 23s
322:	learn: 0.6978306	total: 39.6s	remaining: 1m 23s
323:	learn: 0.6966085	total: 39.8s	remaining: 1m 22s
324:	learn: 0.6955676	total: 39.9s	remaining: 1m 22s
325:	learn: 0.6944006	total: 40s	remaining: 1m 22s
326:	learn: 0.6931250	total: 40.1s	remaining: 1m 22s
327:	learn: 0.6919848	total: 40.2s	remaining: 1m 22s
328:	learn: 0.6906784	total: 40.3s	remaining: 1m 22s
329:	learn: 0.6895344	total: 40.5s	remaining: 1m 22s
330:	learn: 0.6885689	total: 40.6s	remaining: 1m 21s
331:	learn: 0.6873130	total: 40.7s	remaining: 1m 21s
332:	learn: 0.6861141	total: 40.8s	remaining: 1m 2

470:	learn: 0.5675849	total: 59.1s	remaining: 1m 6s
471:	learn: 0.5668676	total: 59.2s	remaining: 1m 6s
472:	learn: 0.5664464	total: 59.4s	remaining: 1m 6s
473:	learn: 0.5657995	total: 59.5s	remaining: 1m 6s
474:	learn: 0.5651747	total: 59.6s	remaining: 1m 5s
475:	learn: 0.5645185	total: 59.8s	remaining: 1m 5s
476:	learn: 0.5637920	total: 59.9s	remaining: 1m 5s
477:	learn: 0.5631673	total: 1m	remaining: 1m 5s
478:	learn: 0.5625567	total: 1m	remaining: 1m 5s
479:	learn: 0.5619586	total: 1m	remaining: 1m 5s
480:	learn: 0.5613229	total: 1m	remaining: 1m 5s
481:	learn: 0.5607712	total: 1m	remaining: 1m 5s
482:	learn: 0.5601991	total: 1m	remaining: 1m 4s
483:	learn: 0.5595704	total: 1m	remaining: 1m 4s
484:	learn: 0.5589433	total: 1m 1s	remaining: 1m 4s
485:	learn: 0.5583313	total: 1m 1s	remaining: 1m 4s
486:	learn: 0.5577672	total: 1m 1s	remaining: 1m 4s
487:	learn: 0.5572392	total: 1m 1s	remaining: 1m 4s
488:	learn: 0.5567022	total: 1m 1s	remaining: 1m 4s
489:	learn: 0.5560489	total: 1m 1

629:	learn: 0.4853213	total: 1m 18s	remaining: 46.4s
630:	learn: 0.4848966	total: 1m 19s	remaining: 46.2s
631:	learn: 0.4844843	total: 1m 19s	remaining: 46.1s
632:	learn: 0.4840515	total: 1m 19s	remaining: 46s
633:	learn: 0.4836379	total: 1m 19s	remaining: 45.9s
634:	learn: 0.4831806	total: 1m 19s	remaining: 45.7s
635:	learn: 0.4827993	total: 1m 19s	remaining: 45.6s
636:	learn: 0.4823930	total: 1m 19s	remaining: 45.5s
637:	learn: 0.4819514	total: 1m 19s	remaining: 45.3s
638:	learn: 0.4815309	total: 1m 20s	remaining: 45.2s
639:	learn: 0.4810192	total: 1m 20s	remaining: 45.1s
640:	learn: 0.4805044	total: 1m 20s	remaining: 45s
641:	learn: 0.4801135	total: 1m 20s	remaining: 44.8s
642:	learn: 0.4797255	total: 1m 20s	remaining: 44.7s
643:	learn: 0.4793414	total: 1m 20s	remaining: 44.6s
644:	learn: 0.4788657	total: 1m 20s	remaining: 44.5s
645:	learn: 0.4784010	total: 1m 20s	remaining: 44.3s
646:	learn: 0.4780021	total: 1m 21s	remaining: 44.2s
647:	learn: 0.4775624	total: 1m 21s	remaining: 44.

785:	learn: 0.4271776	total: 1m 38s	remaining: 26.9s
786:	learn: 0.4267286	total: 1m 39s	remaining: 26.8s
787:	learn: 0.4263428	total: 1m 39s	remaining: 26.7s
788:	learn: 0.4260304	total: 1m 39s	remaining: 26.6s
789:	learn: 0.4257628	total: 1m 39s	remaining: 26.4s
790:	learn: 0.4255451	total: 1m 39s	remaining: 26.3s
791:	learn: 0.4251694	total: 1m 39s	remaining: 26.2s
792:	learn: 0.4248121	total: 1m 39s	remaining: 26.1s
793:	learn: 0.4244553	total: 1m 39s	remaining: 25.9s
794:	learn: 0.4241608	total: 1m 40s	remaining: 25.8s
795:	learn: 0.4237371	total: 1m 40s	remaining: 25.7s
796:	learn: 0.4233553	total: 1m 40s	remaining: 25.5s
797:	learn: 0.4230895	total: 1m 40s	remaining: 25.4s
798:	learn: 0.4228081	total: 1m 40s	remaining: 25.3s
799:	learn: 0.4223731	total: 1m 40s	remaining: 25.2s
800:	learn: 0.4220443	total: 1m 40s	remaining: 25s
801:	learn: 0.4217481	total: 1m 40s	remaining: 24.9s
802:	learn: 0.4213185	total: 1m 40s	remaining: 24.8s
803:	learn: 0.4209811	total: 1m 41s	remaining: 2

940:	learn: 0.3851290	total: 1m 59s	remaining: 7.5s
941:	learn: 0.3848070	total: 1m 59s	remaining: 7.37s
942:	learn: 0.3846408	total: 1m 59s	remaining: 7.24s
943:	learn: 0.3843416	total: 1m 59s	remaining: 7.12s
944:	learn: 0.3841720	total: 2m	remaining: 6.99s
945:	learn: 0.3839481	total: 2m	remaining: 6.86s
946:	learn: 0.3837411	total: 2m	remaining: 6.73s
947:	learn: 0.3835016	total: 2m	remaining: 6.61s
948:	learn: 0.3833160	total: 2m	remaining: 6.48s
949:	learn: 0.3831193	total: 2m	remaining: 6.35s
950:	learn: 0.3827586	total: 2m	remaining: 6.22s
951:	learn: 0.3825955	total: 2m	remaining: 6.1s
952:	learn: 0.3824094	total: 2m 1s	remaining: 5.97s
953:	learn: 0.3821897	total: 2m 1s	remaining: 5.84s
954:	learn: 0.3820229	total: 2m 1s	remaining: 5.71s
955:	learn: 0.3817545	total: 2m 1s	remaining: 5.59s
956:	learn: 0.3814691	total: 2m 1s	remaining: 5.46s
957:	learn: 0.3811719	total: 2m 1s	remaining: 5.33s
958:	learn: 0.3809545	total: 2m 1s	remaining: 5.21s
959:	learn: 0.3807840	total: 2m 1s

In [ ]:
# Post processing the results (Not getting any benefit from this, not sure why)
preds_val = model.predict(testdata)
preds_val=pd.DataFrame(preds_val)
preds_val.reset_index(inplace=True)
preds_df=pd.DataFrame(preds_final)
preds_df.reset_index(inplace=True)
post_process=preds_df.merge(preds_val, on='index')
t=testdata[['Team1', 'Team2']]
t=t.reset_index(drop=True)
t.reset_index(inplace=True)
post_process=post_process.merge(t,on='index' )
post_process.drop(['index'],inplace=True, axis=1)
post_process.columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,'predicted_val','Team1','Team2']
post_process['predicted_val']=post_process['predicted_val'].astype(int)
post_process.head()


In [ ]:

def postprocess_predictions(data):
    for x in range (0,data.shape[0]):
        max_prob_index=data.loc[[x],['predicted_val']].values[0][0]
        t1=data.loc[[x],['Team1']].values[0][0]
        t2=data.loc[[x],['Team2']].values[0][0]
        
        predicted_val=data.loc[[x],['predicted_val']].values[0][0]
        predicted_prob=data.loc[[x],[predicted_val]].values[0][0]
        t1val=data.loc[[x],[t1]].values[0][0]
        t2val=data.loc[[x],[t2]].values[0][0]
        if t1 != predicted_val and t2 != predicted_val:
            if t1val>t2val:
                data.loc[[x],[t1]]=predicted_prob
                data.loc[[x],[t2]]=1-predicted_prob
                for y in range(0,16):
                    if y!=t1 and y!=t2:
                        data.loc[[x],[y]]=0
                
                
            else:
                data.loc[[x],[t1]]=1-predicted_prob
                data.loc[[x],[t2]]=predicted_prob
                for y in range(0,16):
                    if y!=t1 and y!=t2:
                        data.loc[[x],[y]]=0
            
    return(data)

p=postprocess_predictions(post_process)
p.head()
p.drop(['predicted_val','Team1','Team2'], inplace=True, axis=1)
p.to_excel('q8.xlsx',index=False)